In [2]:
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import h5py
import numpy as np
import glob
import shutil
matplotlib.rcParams['figure.figsize'] = [15, 15]
pd.options.mode.chained_assignment = None

# Link each TNEID to a histological type

In [3]:
path_review_agg_by_patient = pd.read_excel('/home/mathiane/LNENWork/ESMOOpenFinalScripts/data/PathologyClinicalDataForYouenn.xlsx', index_col=0, skiprows=[0],
                            engine='openpyxl')

In [4]:
path_review_agg_by_patient.head()

,TNEID,material,final_diagnosis,AC_score,necrosis_bool,necrosis_scores,mean_mitotic_count,mean_ki67_manual_count_in_percent,mean_pph3_count,cytological_atypia,...,census_date,census_status,overall_survival_years,overall_survival_months,PFS_census_date,PFS_census_status,PFS_years,PFS_months,Notes,Molecular_cluster
case_number,,,,,,,,,,,,,,,,,,,,,
1,TNE0056,HES,TC,2,YES,3,1.50,3.33,1.00,NO,...,2017-02-01,alive,14.671233,176.054795,NaN,NaN,NaN,NaN,NaN,NaN
1,TNE1552,Ki67,TC,2,YES,3,1.50,3.33,1.00,NO,...,2017-02-01,alive,14.671233,176.054795,NaN,NaN,NaN,NaN,NaN,NaN
1,TNE1553,pph3,TC,2,YES,3,1.50,3.33,1.00,NO,...,2017-02-01,alive,14.671233,176.054795,NaN,NaN,NaN,NaN,NaN,NaN
2,TNE0057,HES,TC,1,NO,0,0.67,3.17,0.83,NO,...,2019-05-13,alive,12.013699,144.164384,NaN,NaN,NaN,NaN,NaN,CaB
2,TNE1582,Ki67,TC,1,NO,0,0.67,3.17,0.83,NO,...,2019-05-13,alive,12.013699,144.164384,NaN,NaN,NaN,NaN,NaN,CaB


In [6]:
final_diag_tneid = path_review_agg_by_patient[path_review_agg_by_patient["material"] == "HES"][["TNEID", "final_diagnosis"]]

In [7]:
final_diag_tneid.value_counts("final_diagnosis")

final_diagnosis
TC              208
AC               50
NETG3/LCNEC       4
NI                2
undetermined      1
dtype: int64

In [11]:
final_diag_tneid = final_diag_tneid[final_diag_tneid["final_diagnosis"].isin(["TC","AC"])].reset_index()

In [13]:
final_diag_tneid = final_diag_tneid[["TNEID", "final_diagnosis"]]

In [14]:
final_diag_tneid.to_csv("final_diag_tneid.csv")

## Redo the experience scalining vector by TNE ID BUT unbalanced

In [15]:
BT_all_proj = pd.read_csv("/home/mathiane/LNENWork/LNEN_Molecular_groups_barlow_twin/projector/projector_concat_path_review.csv")
BT_all_proj  = BT_all_proj.iloc[:,range(1,BT_all_proj.shape[1],1)]
final_diag_tneid = pd.read_csv("final_diag_tneid.csv")
BTProjDiag= BT_all_proj.merge(final_diag_tneid, left_on='tne_id_c',  right_on='TNEID' , how='inner')
print(BT_all_proj.shape, BTProjDiag.shape )
print(BTProjDiag.head())
BTProjTC= BTProjDiag[BTProjDiag["final_diagnosis"].isin(["TC"])]
BTProjAC = BTProjDiag[BTProjDiag["final_diagnosis"].isin(["AC"])]
print(BTProjAC.shape, BTProjTC.shape)
BTProjTC_s = BTProjTC[BTProjTC["tne_id_c"].isin(list(set(BTProjTC["tne_id_c"])))]
BTProjTCAC_s = pd.concat([BTProjAC, BTProjTC_s])

(4126674, 134) (4126674, 137)
         X0        X1        X2        X3        X4        X5        X6  \
0  0.010731 -0.017492 -0.053791  0.006058 -0.040010  0.006106 -0.024915   
1  0.003174 -0.002447 -0.040421  0.000079 -0.030701  0.005880 -0.027878   
2 -0.010324 -0.008172 -0.039857 -0.003609 -0.023241 -0.007832 -0.032735   
3  0.011308 -0.018657 -0.060345  0.015401 -0.023393 -0.007311 -0.022321   
4 -0.000423 -0.025801 -0.052976  0.005961 -0.026089 -0.010742 -0.021492   

         X7        X8        X9  ...      X127               img_id   tne_id  \
0 -0.011729  0.007921 -0.019953  ... -0.016012  TNE1019_30721_19585  TNE1019   
1 -0.006616  0.011590 -0.017226  ...  0.005784  TNE1019_33409_28801  TNE1019   
2 -0.020779  0.005676 -0.015500  ...  0.010942  TNE1019_45313_14209  TNE1019   
3 -0.004438  0.009153 -0.019563  ... -0.010705   TNE1019_9985_23809  TNE1019   
4 -0.003688  0.008236 -0.015210  ... -0.007972  TNE1019_15361_11905  TNE1019   

              img_id_c  tne_id_c      

In [16]:
BTProjTCAC_s.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X127,img_id,tne_id,img_id_c,tne_id_c,x,y,Unnamed: 0,TNEID,final_diagnosis
0,0.010731,-0.017492,-0.053791,0.006058,-0.040010,0.006106,-0.024915,-0.011729,0.007921,-0.019953,...,-0.016012,TNE1019_30721_19585,TNE1019,TNE1019_30721_19585,TNE1019,30721,19585,123,TNE1019,AC
1,0.003174,-0.002447,-0.040421,0.000079,-0.030701,0.005880,-0.027878,-0.006616,0.011590,-0.017226,...,0.005784,TNE1019_33409_28801,TNE1019,TNE1019_33409_28801,TNE1019,33409,28801,123,TNE1019,AC
2,-0.010324,-0.008172,-0.039857,-0.003609,-0.023241,-0.007832,-0.032735,-0.020779,0.005676,-0.015500,...,0.010942,TNE1019_45313_14209,TNE1019,TNE1019_45313_14209,TNE1019,45313,14209,123,TNE1019,AC
3,0.011308,-0.018657,-0.060345,0.015401,-0.023393,-0.007311,-0.022321,-0.004438,0.009153,-0.019563,...,-0.010705,TNE1019_9985_23809,TNE1019,TNE1019_9985_23809,TNE1019,9985,23809,123,TNE1019,AC
4,-0.000423,-0.025801,-0.052976,0.005961,-0.026089,-0.010742,-0.021492,-0.003688,0.008236,-0.015210,...,-0.007972,TNE1019_15361_11905,TNE1019,TNE1019_15361_11905,TNE1019,15361,11905,123,TNE1019,AC


In [17]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode="a"):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(
                key,
                shape=data_shape,
                maxshape=maxshape,
                chunks=chunk_shape,
                dtype=data_type,
            )
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0] :] = val
    file.close()
    return output_path

In [19]:
root_data = "/home/mathiane/LNENWork/data_RoFormerMIL_TCAC_norm"
os.makedirs(root_data, exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "TC"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "AC"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "TC"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "AC"), exist_ok=True)

for tneid in set(BTProjTCAC_s["tne_id_c"]):
    print("Tneid = ", tneid)
    BTProjTCAC_c = BTProjTCAC_s[BTProjTCAC_s["tne_id_c"] == tneid]
    BTProjTCAC_c = BTProjTCAC_c.sort_values(by=["x","y"])
    BTProjTCAC_c["x_rank"] = BTProjTCAC_c["x"].rank(method="min")-1
    BTProjTCAC_c["y_rank"] = BTProjTCAC_c["y"].rank(method="min")-1
    BTProjTCAC_c["x_rank"] = BTProjTCAC_c["x_rank"].astype(int)
    BTProjTCAC_c["y_rank"] = BTProjTCAC_c["y_rank"].astype(int)
    
    BTProjTCAC_c_features =  BTProjTCAC_c.iloc[:,0:128]
    BTProjTCAC_c_features=(BTProjTCAC_c_features-BTProjTCAC_c_features.mean(axis=0))/BTProjTCAC_c_features.std(axis=0)
    # print("BT normalized_df " , normalized_df.shape, normalized_df.head)
    # print("BT BTProjArchCaA1CaA2_features std " , BTProjArchCaA1CaA2_features.mean(), BTProjArchCaA1CaA2_features.std())
    # mean_over_col = BTProjArchCaA1CaA2_features.mean()
    # std_over_col=  BTProjArchCaA1CaA2_features.std()
    BTProjTCAC_c_coords = BTProjTCAC_c[["img_id_c", "x","y", "x_rank", "y_rank"]]
    BTProjTCAC_c_coords_for_export = BTProjTCAC_c_coords[["x_rank", "y_rank"]]
    #print(BTProjArchCaA1CaA2_coords.head())
    
    type_c = list(BTProjTCAC_c["final_diagnosis"])[0]
    print(type_c)
    asset_dict_tumor = {   
        "coords": BTProjTCAC_c_coords_for_export.to_numpy(),
        "features": BTProjTCAC_c_features.to_numpy(dtype='float32'),
            }
    
    
    h5_file_tumor = os.path.join(root_data, "features_h5_files", type_c, tneid + ".h5" )
    save_hdf5(h5_file_tumor, asset_dict_tumor, attr_dict=None, mode = "w")
    
    coords_tumor_filename = os.path.join(root_data, "coords_table", type_c, tneid + "coords.csv" )
    BTProjTCAC_c_coords.to_csv(coords_tumor_filename, index=False )
    
    print("___________________________________________________ \n")

Tneid =  TNE1024
TC
___________________________________________________ 

Tneid =  TNE1080
TC
___________________________________________________ 

Tneid =  TNE0057
TC
___________________________________________________ 

Tneid =  TNE0408
AC
___________________________________________________ 

Tneid =  TNE1367
TC
___________________________________________________ 

Tneid =  TNE1095
TC
___________________________________________________ 

Tneid =  TNE0006
TC
___________________________________________________ 

Tneid =  TNE0018
TC
___________________________________________________ 

Tneid =  TNE1023
TC
___________________________________________________ 

Tneid =  TNE1438
TC
___________________________________________________ 

Tneid =  TNE1906
TC
___________________________________________________ 

Tneid =  TNE0031
TC
___________________________________________________ 

Tneid =  TNE0987
TC
___________________________________________________ 

Tneid =  TNE0877
AC
__________________

In [20]:
# Create slides_format slides
case_id = []
slide_id = []
label_id = []
label = []
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "TC")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("TC")
        label.append("TC")
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "AC")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("AC")
        label.append("AC")
slides_format = pd.DataFrame({"case_id":case_id,
                              "slide_id": slide_id,
                              "label_id": label_id,
                              "label": label})
print(slides_format.head())
os.makedirs(os.path.join(root_data, "dataRoFormer"), exist_ok=True)

slides_format.to_csv( os.path.join(root_data, "dataRoFormer", "slides_format.csv"))


   case_id slide_id label_id label
0  TNE1794  TNE1794       TC    TC
1  TNE0993  TNE0993       TC    TC
2  TNE1027  TNE1027       TC    TC
3  TNE0872  TNE0872       TC    TC
4  TNE0890  TNE0890       TC    TC


## Norm and roughly balanced

In [21]:
BT_all_proj = pd.read_csv("/home/mathiane/LNENWork/LNEN_Molecular_groups_barlow_twin/projector/projector_concat_path_review.csv")
BT_all_proj  = BT_all_proj.iloc[:,range(1,BT_all_proj.shape[1],1)]
final_diag_tneid = pd.read_csv("final_diag_tneid.csv")
BTProjDiag= BT_all_proj.merge(final_diag_tneid, left_on='tne_id_c',  right_on='TNEID' , how='inner')
print(BT_all_proj.shape, BTProjDiag.shape )
print(BTProjDiag.head())
BTProjTC= BTProjDiag[BTProjDiag["final_diagnosis"].isin(["TC"])]
BTProjAC = BTProjDiag[BTProjDiag["final_diagnosis"].isin(["AC"])]
print(BTProjAC.shape, BTProjTC.shape)
BTProjTC_s = BTProjTC[BTProjTC["tne_id_c"].isin(list(set(BTProjTC["tne_id_c"]))[:60])]
BTProjTCAC_s = pd.concat([BTProjAC, BTProjTC_s])

(4126674, 134) (4126674, 137)
         X0        X1        X2        X3        X4        X5        X6  \
0  0.010731 -0.017492 -0.053791  0.006058 -0.040010  0.006106 -0.024915   
1  0.003174 -0.002447 -0.040421  0.000079 -0.030701  0.005880 -0.027878   
2 -0.010324 -0.008172 -0.039857 -0.003609 -0.023241 -0.007832 -0.032735   
3  0.011308 -0.018657 -0.060345  0.015401 -0.023393 -0.007311 -0.022321   
4 -0.000423 -0.025801 -0.052976  0.005961 -0.026089 -0.010742 -0.021492   

         X7        X8        X9  ...      X127               img_id   tne_id  \
0 -0.011729  0.007921 -0.019953  ... -0.016012  TNE1019_30721_19585  TNE1019   
1 -0.006616  0.011590 -0.017226  ...  0.005784  TNE1019_33409_28801  TNE1019   
2 -0.020779  0.005676 -0.015500  ...  0.010942  TNE1019_45313_14209  TNE1019   
3 -0.004438  0.009153 -0.019563  ... -0.010705   TNE1019_9985_23809  TNE1019   
4 -0.003688  0.008236 -0.015210  ... -0.007972  TNE1019_15361_11905  TNE1019   

              img_id_c  tne_id_c      

In [22]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode="a"):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(
                key,
                shape=data_shape,
                maxshape=maxshape,
                chunks=chunk_shape,
                dtype=data_type,
            )
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0] :] = val
    file.close()
    return output_path

In [23]:
root_data = "/home/mathiane/LNENWork/data_RoFormerMIL_TCAC_norm_balanced"
os.makedirs(root_data, exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "TC"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "AC"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "TC"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "AC"), exist_ok=True)

for tneid in set(BTProjTCAC_s["tne_id_c"]):
    print("Tneid = ", tneid)
    BTProjTCAC_c = BTProjTCAC_s[BTProjTCAC_s["tne_id_c"] == tneid]
    BTProjTCAC_c = BTProjTCAC_c.sort_values(by=["x","y"])
    BTProjTCAC_c["x_rank"] = BTProjTCAC_c["x"].rank(method="min")-1
    BTProjTCAC_c["y_rank"] = BTProjTCAC_c["y"].rank(method="min")-1
    BTProjTCAC_c["x_rank"] = BTProjTCAC_c["x_rank"].astype(int)
    BTProjTCAC_c["y_rank"] = BTProjTCAC_c["y_rank"].astype(int)
    
    BTProjTCAC_c_features =  BTProjTCAC_c.iloc[:,0:128]
    BTProjTCAC_c_features=(BTProjTCAC_c_features-BTProjTCAC_c_features.mean(axis=0))/BTProjTCAC_c_features.std(axis=0)
    # print("BT normalized_df " , normalized_df.shape, normalized_df.head)
    # print("BT BTProjArchCaA1CaA2_features std " , BTProjArchCaA1CaA2_features.mean(), BTProjArchCaA1CaA2_features.std())
    # mean_over_col = BTProjArchCaA1CaA2_features.mean()
    # std_over_col=  BTProjArchCaA1CaA2_features.std()
    BTProjTCAC_c_coords = BTProjTCAC_c[["img_id_c", "x","y", "x_rank", "y_rank"]]
    BTProjTCAC_c_coords_for_export = BTProjTCAC_c_coords[["x_rank", "y_rank"]]
    #print(BTProjArchCaA1CaA2_coords.head())
    
    type_c = list(BTProjTCAC_c["final_diagnosis"])[0]
    print(type_c)
    asset_dict_tumor = {   
        "coords": BTProjTCAC_c_coords_for_export.to_numpy(),
        "features": BTProjTCAC_c_features.to_numpy(dtype='float32'),
            }
    
    
    h5_file_tumor = os.path.join(root_data, "features_h5_files", type_c, tneid + ".h5" )
    save_hdf5(h5_file_tumor, asset_dict_tumor, attr_dict=None, mode = "w")
    
    coords_tumor_filename = os.path.join(root_data, "coords_table", type_c, tneid + "coords.csv" )
    BTProjTCAC_c_coords.to_csv(coords_tumor_filename, index=False )
    
    print("___________________________________________________ \n")

Tneid =  TNE1024
TC
___________________________________________________ 

Tneid =  TNE1080
TC
___________________________________________________ 

Tneid =  TNE0057
TC
___________________________________________________ 

Tneid =  TNE0408
AC
___________________________________________________ 

Tneid =  TNE1367
TC
___________________________________________________ 

Tneid =  TNE1095
TC
___________________________________________________ 

Tneid =  TNE0006
TC
___________________________________________________ 

Tneid =  TNE0018
TC
___________________________________________________ 

Tneid =  TNE1023
TC
___________________________________________________ 

Tneid =  TNE1438
TC
___________________________________________________ 

Tneid =  TNE1906
TC
___________________________________________________ 

Tneid =  TNE0031
TC
___________________________________________________ 

Tneid =  TNE0987
TC
___________________________________________________ 

Tneid =  TNE0877
AC
__________________

In [24]:
# Create slides_format slides
case_id = []
slide_id = []
label_id = []
label = []
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "TC")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("TC")
        label.append("TC")
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "AC")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("AC")
        label.append("AC")
slides_format = pd.DataFrame({"case_id":case_id,
                              "slide_id": slide_id,
                              "label_id": label_id,
                              "label": label})
print(slides_format.head())
os.makedirs(os.path.join(root_data, "dataRoFormer"), exist_ok=True)

slides_format.to_csv( os.path.join(root_data, "dataRoFormer", "slides_format.csv"))


   case_id slide_id label_id label
0  TNE1794  TNE1794       TC    TC
1  TNE1430  TNE1430       TC    TC
2  TNE0263  TNE0263       TC    TC
3  TNE1080  TNE1080       TC    TC
4  TNE2141  TNE2141       TC    TC
